In [3]:
import tkinter as tk
import socket
import threading

def send_message():
    message = message_entry.get()
    mode = mode_var.get()

    # Creating a socket connection
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = ('localhost', 8000)
    client_socket.connect(server_address)

    # Sending the message and mode of encoding to the other client
    client_socket.sendall(f"{mode}:{message}".encode())

    # Receiving the response from the other client
    response = client_socket.recv(1024).decode()

    # Displaying the response
    response_label.configure(text=f"Response: {response}")

    # Closing the socket connection
    client_socket.close()

def receive_thread():
    # Creating a socket connection
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = ('localhost', 8001)
    server_socket.bind(server_address)
    server_socket.listen(1)
    
    while True:
        # Accepting a client connection
        client_socket, client_address = server_socket.accept()
        
        # Receiving the message and mode of encoding from the other client
        data = client_socket.recv(1024).decode()
        mode, message = data.split(':')

        # Displaying the received message and mode of encoding
        received_label.configure(text=f"Received message: {message}")
        mode_label.configure(text=f"Mode of encoding: {mode}")

        # Sending a response back to the other client
        response = "Message received successfully!"
        client_socket.sendall(response.encode())

        # Closing the socket connection
        client_socket.close()

# Creating the client UI
client = tk.Tk()
client.title("Client - 2")

# Message input
message_label = tk.Label(client, text="Enter your message:")
message_label.pack()
message_entry = tk.Entry(client)
message_entry.pack()

# Mode of encoding input
mode_label = tk.Label(client, text="Select the mode of encoding:")
mode_label.pack()

# Mode of encoding radio buttons
mode_var = tk.StringVar(value="text")
image_radio = tk.Radiobutton(client, text="Image", variable=mode_var, value="image")
image_radio.pack()
video_radio = tk.Radiobutton(client, text="Video", variable=mode_var, value="video")
video_radio.pack()
text_radio = tk.Radiobutton(client, text="Text", variable=mode_var, value="text")
text_radio.pack()

# Send button
send_button = tk.Button(client, text="Send", command=send_message)
send_button.pack()

# Received message label
received_label = tk.Label(client, text="Received message:")
received_label.pack()

# Response label
response_label = tk.Label(client, text="")
response_label.pack()

# Start the receive thread
receive_thread = threading.Thread(target=receive_thread)
receive_thread.daemon = True
receive_thread.start()

# Start the client UI
client.mainloop()

Exception in thread Thread-7 (receive_thread):
Traceback (most recent call last):
  File "c:\Users\farquleet\anaconda3\envs\myenv\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\farquleet\anaconda3\envs\myenv\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\farquleet\AppData\Local\Temp\ipykernel_11376\2569921418.py", line 30, in receive_thread
OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted


In [26]:
import tkinter as tk
from tkinter import filedialog
import socket
import threading
import cv2
import numpy as np

def send_message():
    mode = mode_var.get()

    # Open file dialog based on the selected mode
    file_types = [("Image Files", "*.png *.jpg *.jpeg"), ("Video Files", "*.mp4 *.avi *.mov"), ("Text Files", "*.txt")]
    file_path = filedialog.askopenfilename(filetypes=file_types)
    
    if file_path:
        # Read the file content
        with open(file_path, 'rb') as file:
            message = file.read()

        # Creating a socket connection
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_address = ('localhost', 8000)
        client_socket.connect(server_address)

        # Sending the message and mode of encoding to the other client
        client_socket.sendall(f"{mode}:{message}".encode())

        # Receiving the response from the other client
        response = client_socket.recv(1024).decode()

        # Displaying the response
        response_label.configure(text=f"Response: {response}")

        # Closing the socket connection
        client_socket.close()
    else:
        response_label.configure(text="No file selected.")

def receive_thread():
    # Creating a socket connection
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = ('localhost', 8001)
    server_socket.bind(server_address)
    server_socket.listen(1)
    

    # Accepting a client connection
    client_socket, client_address = server_socket.accept()
    
    # Receiving the message and mode of encoding from the other client
    data = client_socket.recv(1024).decode()
    message= data
    mode = "image"
    # mode, message = data.split(':')

    # Displaying the received message and mode of encoding
    received_label.configure(text=f"Received message: {message}")
    mode_label.configure(text=f"Mode of encoding: {mode}")

    # Saving the received file based on the mode
    save_file(mode, message)

    # Sending a response back to the other client
    response = "Message received successfully!"
    client_socket.sendall(response.encode())

    # Closing the socket connection
    client_socket.close()

def save_file(mode, message):
    if mode == "image":
        file_extension = ".png"
        message = np.array(message)
        print(type(message))
        cv2.imwrite("received_file.png", message)
        
    elif mode == "video":
        file_extension = ".mp4"
    else:
        file_extension = ".txt"
    
    # # Save the file with a unique name
    # file_name = f"received_file{file_extension}"
    # with open(file_name, 'wb') as file:
    #     file.write(message)

# Creating the client UI
client = tk.Tk()
client.title("Client - 2")

# Message input
message_label = tk.Label(client, text="Select a file:")
message_label.pack()

# Mode of encoding input
mode_label = tk.Label(client, text="Select the mode of encoding:")
mode_label.pack()

# Mode of encoding radio buttons
mode_var = tk.StringVar(value="text")
image_radio = tk.Radiobutton(client, text="Image", variable=mode_var, value="image")
image_radio.pack()
video_radio = tk.Radiobutton(client, text="Video", variable=mode_var, value="video")
video_radio.pack()
text_radio = tk.Radiobutton(client, text="Text", variable=mode_var, value="text")
text_radio.pack()

# Send button
send_button = tk.Button(client, text="Send", command=send_message)
send_button.pack()

# Received message label
received_label = tk.Label(client, text="Received message:")
received_label.pack()

# Response label
response_label = tk.Label(client, text="")
response_label.pack()

# Start the receive thread
receive_thread = threading.Thread(target=receive_thread)
receive_thread.daemon = True
receive_thread.start()

# Start the client UI
client.mainloop()

Exception in thread Thread-30 (receive_thread):
Traceback (most recent call last):
  File "c:\Users\farquleet\anaconda3\envs\myenv\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\farquleet\anaconda3\envs\myenv\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\farquleet\AppData\Local\Temp\ipykernel_11376\2892051017.py", line 61, in receive_thread
  File "C:\Users\farquleet\AppData\Local\Temp\ipykernel_11376\2892051017.py", line 75, in save_file
cv2.error: OpenCV(4.8.1) :-1: error: (-5:Bad argument) in function 'imwrite'
> Overload resolution failed:
>  - img data type = 19 is not supported
>  - Expected Ptr<cv::UMat> for argument 'img'



<class 'numpy.ndarray'>
